In [8]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [9]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [10]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [11]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [12]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [13]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [14]:
tf.reset_default_graph()

# Exercise 12

In [16]:
from sklearn.datasets import make_moons

moon_X, moon_y = make_moons()

In [17]:
moon_X.shape

(100, 2)

In [18]:
moon_y

array([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [19]:
moon_y.shape

(100,)

In [15]:
def logistic_regression(w, b, x):
    with tf.name_scope("logistric_regression") as scope:
        weighted_input_plus_bias = w * x + b
        sigmoid = 1 / (1 + tf.exp(-weighted_input_plus_bias))
        return sigmoid

In [67]:
number_of_features = moon_X.shape[1]
w = tf.Variable(tf.random_normal([number_of_features], name="weights"))
b = tf.Variable(0.0, name="bias")
x = tf.placeholder(tf.float32, shape=(None, number_of_features), name="input")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
output = logistic_regression(w, b, x)

error = output - y

mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.2)
training_op = optimizer.minimize(mse)
correctness = tf.reduce_mean(tf.cast(tf.abs(error) < 0.1, tf.float32)) * 100

In [72]:
import numpy as np

epochs = 500
batch_size = 10
n_batches = int(np.ceil(moon_X.shape[0] / batch_size))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/logistic_regression.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', mse)
correctness_summary = tf.summary.scalar("correctness", correctness)
summary_op = tf.summary.merge([mse_summary, correctness_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    #saver.restore(sess, interim_checkpoint_path)
    
    for epoch in range(epochs):
        for batch_index in range(n_batches):
            batch_X = moon_X[batch_index * batch_size:(batch_index + 1) * batch_size]
            #print("batch_X shape:", batch_X.shape)
            batch_y = moon_y[batch_index * batch_size:(batch_index + 1) * batch_size].reshape((-1, 1))
            #print("batch_Y shape:", batch_y.shape)
            if batch_index % 10 == 0:
                summary_str = summary_op.eval(feed_dict={x: batch_X, y: batch_y})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            t, m = sess.run([training_op, mse], feed_dict={x: batch_X, y: batch_y})
            
            #print("Batch performance: ", batch_correctness)
            if batch_index == 0: print("Mean squared error:", m)
        if epoch % 100 == 0:
            save_path = saver.save(sess, interim_checkpoint_path)

    save_path = saver.save(sess, "./checkpoints/logistic_regression_final.ckpt")

Mean squared error: 0.197826
Mean squared error: 0.191941
Mean squared error: 0.187382
Mean squared error: 0.183767
Mean squared error: 0.180841
Mean squared error: 0.178429
Mean squared error: 0.17641
Mean squared error: 0.174695
Mean squared error: 0.173222
Mean squared error: 0.171943
Mean squared error: 0.170823
Mean squared error: 0.169834
Mean squared error: 0.168956
Mean squared error: 0.168173
Mean squared error: 0.167469
Mean squared error: 0.166836
Mean squared error: 0.166264
Mean squared error: 0.165746
Mean squared error: 0.165276
Mean squared error: 0.164849
Mean squared error: 0.16446
Mean squared error: 0.164106
Mean squared error: 0.163784
Mean squared error: 0.16349
Mean squared error: 0.163223
Mean squared error: 0.162979
Mean squared error: 0.162757
Mean squared error: 0.162556
Mean squared error: 0.162373
Mean squared error: 0.162207
Mean squared error: 0.162057
Mean squared error: 0.161922
Mean squared error: 0.1618
Mean squared error: 0.161691
Mean squared error:

Mean squared error: 0.170017
Mean squared error: 0.170034
Mean squared error: 0.17005
Mean squared error: 0.170067
Mean squared error: 0.170083
Mean squared error: 0.1701
Mean squared error: 0.170116
Mean squared error: 0.170132
Mean squared error: 0.170149
Mean squared error: 0.170165
Mean squared error: 0.170181
Mean squared error: 0.170197
Mean squared error: 0.170213
Mean squared error: 0.170228
Mean squared error: 0.170244
Mean squared error: 0.17026
Mean squared error: 0.170275
Mean squared error: 0.17029
Mean squared error: 0.170306
Mean squared error: 0.170321
Mean squared error: 0.170336
Mean squared error: 0.170351
Mean squared error: 0.170366
Mean squared error: 0.170381
Mean squared error: 0.170396
Mean squared error: 0.170411
Mean squared error: 0.170426
Mean squared error: 0.17044
Mean squared error: 0.170455
Mean squared error: 0.170469
Mean squared error: 0.170484
Mean squared error: 0.170498
Mean squared error: 0.170512
Mean squared error: 0.170526
Mean squared error: 